Air stagnation indices over the Euro-Mediterranean region
=====

This air stagnation index (ASI) was introduced by Wang et al. (2016, 2018), using meteorological and air quality data over 
China, Europe and the United States. It looks for meteorological conditions conducive to elevated concentrations of PM2.5 
(particulate matter with aerodynamic diameter ≤ 2.5 µm). For that purpose, it uses near-surface `wind speed` (W), 
atmospheric `boundary layer height` (BLH) and precipitation as indices of atmospheric horizontal dispersion capability, 
vertical mixing and wet deposition, respectively.

    Copyright (C) 2020  Carlos Ordóñez García
    Copyright (C) 2020  José Manuel Garrido Pérez
    Copyright (C) 2020  Javier Ruano Ruano

    This program is free software: you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation, either version 3 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License
    along with this program.  If not, see <https://www.gnu.org/licenses/>.



Wang's ASI has been computed over land areas by using meteorological fields from the ERA5 reanalysis. 
A grid cell is considered as stagnant on a given day if two conditions are simultaneously met:
BLH at 12UTC is below that computed from the equations above using the daily average wind speed at 10 m, 
and the daily accumulated precipitation is below 1 mm.

* References  
 + Wang, X., Dickinson, R. E., Su, L., Zhou, C., Wang, K., 2018. PM2.5 pollution in China and how it has been exacerbated by terrain and meteorological conditions. B. Am. Meteorol. Soc. 99(1), 105-119. https://doi.org/10.1175/BAMS-D-16-0301.1.

 + Wang, X., Wang, K., Su, L., 2016. Contribution of atmospheric diffusion conditions to the recent improvement in air quality in China. Sci. Rep. 6, 36404. https://doi.org/10.1038/srep36404.

Website : http://steady-ucm.org

Github: https://github.com/JavierRuano/ASI_Steady


Loading the libraries
==

In [4]:

import xarray as xr
import numpy as np
import itertools
import pandas as pd

Dataset 
==
Loading datasets obtained with Era5 copernicus, with a frequency of `every six hours` except for
boundary layer height.
https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=overview

In [5]:
Boundary_Layer_Hei=xr.open_mfdataset('Dataset/*boundary_layer_height*.nc',parallel=True,combine='nested', concat_dim='time')
Surface_Wind_Speed_u10=xr.open_mfdataset('Dataset/*10m_u_component_of_wind*.nc',parallel=True,combine='nested', concat_dim='time')
Surface_Wind_Speed_v10=xr.open_mfdataset('Dataset/*10m_v_component_of_wind*.nc',parallel=True,combine='nested', concat_dim='time')
total_precipitation=xr.open_mfdataset('Dataset/*total_precipitation*.nc',parallel=True,combine='nested', concat_dim='time')

In [8]:
total_precipitation.time[0:8]


<xarray.DataArray 'time' (time: 8)>
array(['2020-11-30T00:00:00.000000000', '2020-11-30T06:00:00.000000000',
       '2020-11-30T12:00:00.000000000', '2020-11-30T18:00:00.000000000',
       '2020-12-01T00:00:00.000000000', '2020-12-01T06:00:00.000000000',
       '2020-12-01T12:00:00.000000000', '2020-12-01T18:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2020-11-30 ... 2020-12-01T18:00:00
Attributes:
    long_name:  time

In [10]:
Surface_Wind_Speed=Surface_Wind_Speed_u10.merge(Surface_Wind_Speed_v10)
Surface_Wind_Speed.time[0:8]

<xarray.DataArray 'time' (time: 8)>
array(['2020-11-30T00:00:00.000000000', '2020-11-30T06:00:00.000000000',
       '2020-11-30T12:00:00.000000000', '2020-11-30T18:00:00.000000000',
       '2020-12-01T00:00:00.000000000', '2020-12-01T06:00:00.000000000',
       '2020-12-01T12:00:00.000000000', '2020-12-01T18:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2020-11-30 ... 2020-12-01T18:00:00
Attributes:
    long_name:  time

In [12]:
Boundary_Layer_Hei['time'][0:8]

<xarray.DataArray 'time' (time: 8)>
array(['2020-11-30T00:00:00.000000000', '2020-11-30T06:00:00.000000000',
       '2020-11-30T12:00:00.000000000', '2020-11-30T18:00:00.000000000',
       '2020-12-01T00:00:00.000000000', '2020-12-01T06:00:00.000000000',
       '2020-12-01T12:00:00.000000000', '2020-12-01T18:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2020-11-30 ... 2020-12-01T18:00:00
Attributes:
    long_name:  time

calculation wind module
==


In [14]:
Surface_Wind_Mod=xr.ufuncs.sqrt(Surface_Wind_Speed.u10**2 + Surface_Wind_Speed.v10**2).sel(time=slice(Surface_Wind_Speed['time'][0].values,Surface_Wind_Speed['time'][-3].values))


In [15]:
Surface_Wind_Mod

<xarray.DataArray (time: 126, latitude: 68, longitude: 81)>
dask.array<getitem, shape=(126, 68, 81), dtype=float32, chunksize=(126, 68, 81), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float32 -20.0 -19.25 -18.5 -17.75 ... 38.5 39.25 40.0
  * latitude   (latitude) float32 75.0 74.25 73.5 72.75 ... 26.25 25.5 24.75
  * time       (time) datetime64[ns] 2020-11-30 ... 2020-12-31T06:00:00

Daily indices
==

* calculation of daily indices from four samples per day. 
    + The time index will be used to reduce datasets. First it will take only one of the daily hours and 
    be used as a new index.
    + The np.add.reduceat function was used to generate a new array of values with which to create the new xarrays. DataArray, using the previous latitude and longitude and modifying the time index.

In [32]:
List1=np.arange(len(Surface_Wind_Speed.time.values))[:int((len(Surface_Wind_Speed.time.values)/2)-1)]
List2=np.arange(len(Surface_Wind_Speed.time.values))[:int((len(Surface_Wind_Speed.time.values)/2)-1)]+4

time_Index=Surface_Wind_Speed.time.values[::4]
latitude_Index=Surface_Wind_Speed.latitude.values
longitude_Index=Surface_Wind_Speed.longitude.values


Reduce functions and new DataArray creation
====

In [47]:
Module_Wind_Surface=np.add.reduceat(Surface_Wind_Mod.values,[val for pair in zip(List1, List2) for val in pair])[::4]/4
Sum_Rain=np.add.reduceat(total_precipitation.tp.values,[val for pair in zip(List1, List2) for val in pair])[::4]
Wind_Surface_Array=xr.DataArray(Module_Wind_Surface,coords=[time_Index,latitude_Index,longitude_Index],dims=['time','latitude','longitude'])
Rain_Array=xr.DataArray(Sum_Rain,coords=[time_Index,latitude_Index,longitude_Index],dims=['time','latitude','longitude'])


Reduce BLH to 12:00 each day
==

In [67]:
Boundary_Layer_Hei['blh'][2::4]

<xarray.DataArray 'blh' (time: 32, latitude: 68, longitude: 81)>
dask.array<getitem, shape=(32, 68, 81), dtype=float32, chunksize=(32, 68, 81), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float32 -20.0 -19.25 -18.5 -17.75 ... 38.5 39.25 40.0
  * latitude   (latitude) float32 75.0 74.25 73.5 72.75 ... 26.25 25.5 24.75
  * time       (time) datetime64[ns] 2020-11-30T12:00:00 ... 2020-12-31T12:00:00
Attributes:
    units:      m
    long_name:  Boundary layer height

In [68]:
Boundary_Layer_Hei=Boundary_Layer_Hei['blh'][2::4]

All DataArray have a new daily index

In [20]:
Index=pd.DataFrame(Wind_Surface_Array.time.values,columns=['date'])

In [21]:
Index[0:4]

,date
0,2020-11-30
1,2020-12-01
2,2020-12-02
3,2020-12-03


Wang season formulas
==
Marking which season belongs to every day, and applying a different function as Wang formula.

* Seasons changes
    + Spring: BLH = 3.57 * 103 * exp(-3.35 * W) + 0.352
    + Summer: BLH = 7.66 * 10 * exp(-2.12 * W) + 0.443
    + Autumn: BLH = 1.88 * 104 * exp(-5.15 * W) + 0.440
    + Winter: BLH = 0.759 * exp(-0.6 * W) + 0.264

Before
===

In [33]:
Wind_Surface_Array[0]

<xarray.DataArray (latitude: 68, longitude: 81)>
array([[ 3.2467253,  2.052974 ,  2.2867024, ...,  8.574057 ,  8.590838 ,
         8.587336 ],
       [ 3.782639 ,  2.848401 ,  2.8445928, ...,  9.422142 ,  9.469935 ,
         9.576614 ],
       [ 2.6712303,  3.229898 ,  3.6542242, ..., 10.876069 , 10.712591 ,
        10.829819 ],
       ...,
       [ 7.3016224,  7.0119805,  6.8812037, ...,  2.5106125,  3.7175374,
         5.818311 ],
       [ 6.198871 ,  6.0907345,  5.9739084, ...,  2.6992722,  5.1594086,
         4.982341 ],
       [ 5.3773518,  4.95829  ,  4.634058 , ...,  2.1032858,  3.5709598,
         5.147344 ]], dtype=float32)
Coordinates:
    time       datetime64[ns] 2020-11-30
  * latitude   (latitude) float32 75.0 74.25 73.5 72.75 ... 26.25 25.5 24.75
  * longitude  (longitude) float32 -20.0 -19.25 -18.5 -17.75 ... 38.5 39.25 40.0

After
===

In [35]:
count=0
def define_period(season):
    if(season in [3,4,5]):
        return 1
    elif(season in [6,7,8]):
        return 2
    elif(season in [9,10,11]):
        return 3
    else:
        return 0
count=0
operationSeason=Index['date'].astype('datetime64').dt.month.apply(define_period).values


def s1(b):
    return np.multiply(np.multiply(3.57,1000),np.exp(-3.35*b))+0.352
def s2(b):
    return np.multiply(np.multiply(7.66,10),np.exp(-2.12*b))+0.443
def s3(b):
    return np.multiply(np.multiply(1.88,10000),np.exp(-5.15*b))+0.440
def s4(b):
    return np.multiply(0.759,np.exp(-0.6*b))+0.264
function = {
        "1": s1,
        "2": s2,
        "3": s3,
        "0": s4
    }
for i in operationSeason:
    Wind_Surface_Array[count]=function[str(i)](Wind_Surface_Array[count])
    count+=1

Wind_Surface_Array[0]




<xarray.DataArray (latitude: 68, longitude: 81)>
array([[0.44102913, 0.92132926, 0.5844379 , ..., 0.44      , 0.44      ,
        0.44      ],
       [0.44006515, 0.4480053 , 0.44816384, ..., 0.44      , 0.44      ,
        0.44      ],
       [0.45993608, 0.44112232, 0.4401262 , ..., 0.44      , 0.44      ,
        0.44      ],
       ...,
       [0.44      , 0.44      , 0.44      , ..., 0.48559114, 0.44009107,
        0.44      ],
       [0.44      , 0.44      , 0.44      , ..., 0.45725524, 0.44000006,
        0.44000015],
       [0.44000003, 0.44000015, 0.4400008 , ..., 0.81146175, 0.44019377,
        0.44000006]], dtype=float32)
Coordinates:
    time       datetime64[ns] 2020-11-30
  * latitude   (latitude) float32 75.0 74.25 73.5 72.75 ... 26.25 25.5 24.75
  * longitude  (longitude) float32 -20.0 -19.25 -18.5 -17.75 ... 38.5 39.25 40.0

If we would like to operate blh DataArray with Wind_Surface_Array the time index should be the same. 

In [84]:
Boundary_Layer_Hei

<xarray.DataArray 'blh' (time: 32, latitude: 68, longitude: 81)>
dask.array<getitem, shape=(32, 68, 81), dtype=float32, chunksize=(32, 68, 81), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float32 -20.0 -19.25 -18.5 -17.75 ... 38.5 39.25 40.0
  * latitude   (latitude) float32 75.0 74.25 73.5 72.75 ... 26.25 25.5 24.75
  * time       (time) datetime64[ns] 2020-11-30T12:00:00 ... 2020-12-31T12:00:00
Attributes:
    units:      m
    long_name:  Boundary layer height

Boolean DataArray for Wang Stagnation Index
==

Directly comparing the DataArray to the stagnation conditions and producing a Boolean DataArray,
providing `True` if there is stagnation and `False` opposite case.

In [79]:
(((2*(Rain_Array>0.001)) -(Boundary_Layer_Hei.values<Wind_Surface_Array)*1)>=1).to_dataset(name='ASI_WANG')

<xarray.Dataset>
Dimensions:    (latitude: 68, longitude: 81, time: 32)
Coordinates:
  * time       (time) datetime64[ns] 2020-11-30 2020-12-01 ... 2020-12-31
  * latitude   (latitude) float32 75.0 74.25 73.5 72.75 ... 26.25 25.5 24.75
  * longitude  (longitude) float32 -20.0 -19.25 -18.5 -17.75 ... 38.5 39.25 40.0
Data variables:
    ASI_WANG   (time, latitude, longitude) bool False False ... False False

In [80]:
(((2*(Rain_Array>0.001)) -(Boundary_Layer_Hei.values<Wind_Surface_Array)*1)>=1).to_dataset(name='ASI_WANG')['ASI_WANG']

<xarray.DataArray 'ASI_WANG' (time: 32, latitude: 68, longitude: 81)>
array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ...,  True,  True,  True],
        [False, False, False, ...,  True, False,  True],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False,  True,  True],
        [False, False, False, ...,  True, False, False],
        [False, False, False, ..., False, False, False],
        ...,
...
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ...,  True,  True,  True],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ...,  True,  True,  True],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]])
Coordinates:
  * time       (time) datetime64[ns] 2020-11-30 2020-12-01 ... 2020-12-31
  * latitude   (latitude) float32 75.0 74.25 73.5 72.75 ... 26.25 25.5 24.75
  * longitude  (longitude) float32 -20.0 -19.25 -18.5 -17.75 ... 38.5 39.25 40.0